In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
#nltk.download()

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [3]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [4]:
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')

In [5]:
alice = gutenberg.raw('carroll-alice.txt')

In [6]:
# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)

In [7]:
alice = re.sub(r'CHAPTER .*', '', alice)

In [8]:
alice = text_cleaner(alice)

In [9]:
persuasion = text_cleaner(persuasion)

In [10]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [11]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll
4,"((, when, she, thought, it, over, afterwards, ...",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [12]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [13]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500


,send,flustered,jury,ann,signify,camden,pocket,dead,apparently,marry,...,style,constancy,stile,sign,mile,coaxing,loss,d,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll
4,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,"((, when, she, thought, it, over, afterwards, ...",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [14]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.990088105727

Test set score: 0.921806167401


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with lasso.

In [15]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(2724, 3023) (2724,)
Training set score: 0.963656387665

Test set score: 0.935572687225


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [16]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.919970631424

Test set score: 0.906938325991


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [17]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [18]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [19]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [20]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
done


In [21]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.703466666667


col_0,Austen,Carroll
row_0,,
Austen,1158,21
Carroll,535,161


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 97%.  

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [22]:
# Challenge 0
from sklearn.svm import SVC

# Trying out SVC
# Make a model using SVC here
svc = SVC()
svc.fit(X_train, y_train)

# Model.
print('\nTest set score:', svc.score(X_test, y_test))
svc_predicted = svc.predict(X_test)
pd.crosstab(y_test, svc_predicted)


Test set score: 0.734030837004


col_0,Austen
text_source,
Austen,1333
Carroll,483


In [23]:
# The SVC model did not predict any titles by 'Carroll'

In [24]:
# making more features that take advantage of the spaCy information 
# (include grammar, phrases, POS, etc)
# Will include pos to see how it helps 
def parts_of_speech(text):

#    pos = [token.pos_
#          for token in text
#          if not token.is_punct
#          and not token.is_stop]

    pos = [token.pos_ for token in text]
    return [item for item in pos]

def pos_features(sentences, common_pos):
    d2 = pd.DataFrame(columns=common_pos)
    d2['text_sentence'] = sentences[0]
    d2['text_source'] = sentences[1]
    d2.loc[:, common_pos] = 0
    
    for i, sentence in enumerate(d2['text_sentence']):
        parts = [token.pos_
                for token in sentence
                if (
#                    not token.is_punct
#                    and not token.is_stop
#                    and 
                    token.pos_ in common_pos
                )]
        for part in parts:
            d2.loc[i, part] += 1
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return d2

# Set up the groups.
alicepos = parts_of_speech(alice_doc)
persuasionpos = parts_of_speech(persuasion_doc)

# Combine groups to create a set of parts of speech.
common_pos = set(alicepos + persuasionpos)

In [25]:
# Create our data frame with features. This can take a while to run.
pos_counts = pos_features(sentences, common_pos)
pos_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500


,CCONJ,PART,PUNCT,ADJ,PROPN,PRON,NUM,NOUN,VERB,ADV,X,DET,ADP,INTJ,text_sentence,text_source
0,6,3,9,3,2,3,0,12,13,3,0,5,8,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,2,1,7,7,2,4,0,8,11,7,0,6,8,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,1,1,4,2,4,2,0,2,5,1,0,3,6,2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,3,1,0,1,0,0,2,0,0,0,0,2,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll
4,8,5,17,5,4,13,0,15,19,11,0,11,18,0,"((, when, she, thought, it, over, afterwards, ...",Carroll


In [26]:
# Merge the two dataframes
frames = [word_counts, pos_counts]

combined = pd.concat(frames, axis=1, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
combined.head()

,send,flustered,jury,ann,signify,camden,pocket,dead,apparently,marry,...,NUM,NOUN,VERB,ADV,X,DET,ADP,INTJ,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,12,13,3,0,5,8,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,8,11,7,0,6,8,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,2,5,1,0,3,6,2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,2,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll
4,0,0,0,0,0,0,2,0,0,0,...,0,15,19,11,0,11,18,0,"((, when, she, thought, it, over, afterwards, ...",Carroll


In [27]:
print(np.where(combined.columns == 'text_source'))

(array([3024, 3040], dtype=int64),)


In [28]:
# Since there are two columns for 'text_source,' drop one of them
combined = combined.groupby(level=0, axis=1).last()

In [29]:
print(np.where(combined.columns == 'text_source'))

(array([2709], dtype=int64),)


In [30]:
Y = combined['text_source']
X = np.array(combined.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = lr.fit(X_train, y_train)

print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

Training set score: 0.962922173275

Test set score: 0.941079295154


Adding parts of speech did not improve accuracy (previous was .935)

In [31]:
# Challenge 0
from sklearn.svm import SVC

# Trying out SVC
# Make a model using SVC here
svc = SVC()
svc.fit(X_train, y_train)

# Model.
print('\nTest set score:', svc.score(X_test, y_test))
svc_predicted = svc.predict(X_test)
pd.crosstab(y_test, svc_predicted)


Test set score: 0.752202643172


col_0,Austen,Carroll
text_source,,
Austen,1329,4
Carroll,446,37


SVC did not perform better with the new features

In [32]:
mean_pos = combined[['NOUN','ADJ','INTJ','PUNCT','ADP','PROPN','VERB','DET','NUM','PART','ADV','PRON','X','text_source']].groupby(['text_source']).mean()
                 
print(mean_pos)

                 NOUN       ADJ      INTJ     PUNCT       ADP     PROPN  \
text_source                                                               
Austen       3.746504  2.659030  0.066944  4.224338  3.269563  1.490330   
Carroll      3.240034  1.766751  0.134860  6.027142  2.562341  1.146735   

                 VERB       DET       NUM      PART       ADV      PRON  \
text_source                                                               
Austen       5.342458  2.046712  0.157691  0.764653  2.192800  2.167510   
Carroll      5.296862  2.470738  0.167939  0.698897  2.263783  2.401187   

                    X  
text_source            
Austen       0.000000  
Carroll      0.000848  


In [33]:
# Trying again, but with only the top seven different pos
Y = combined['text_source']
X = combined.drop(['text_sentence','text_source', 'NOUN', 'ADJ', 'INTJ', 'PROPN', 'VERB', 'ADV'], 1)

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = lr.fit(X_train, y_train)

print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

Training set score: 0.965491923642

Test set score: 0.936674008811


In [34]:
d3 = pd.DataFrame()
d3['text_sentence'] = sentences[0]
d3['text_source'] = sentences[1]
d3.loc[:, 'word_counts'] = 0

for i, sentence in enumerate(d3['text_sentence']):
    word_counts = 0
    words = [token
             for token in sentence
             if (
                 not token.is_punct
                 and not token.is_stop
                 
                 )]
    word_counts = len(words)
    d3.loc[i, 'word_counts'] = word_counts
    if i % 500 == 0:
        print('Processing row {}'.format(i))
        
d3.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500


,text_sentence,text_source,word_counts
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,20
1,"(So, she, was, considering, in, her, own, mind...",Carroll,23
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,8
3,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll,4
4,"((, when, she, thought, it, over, afterwards, ...",Carroll,35


In [35]:
# Merge the two dataframes
frames = [combined, d3]

combined2 = pd.concat(frames, axis=1, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)
combined2.head()

,'d,'em,'ll,'m,'s,'ve,-PRON-,1,ADJ,ADP,...,you'd,young,youth,zeal,zealand,zealous,zigzag,text_sentence,text_source,word_counts
0,0,0,0,0,0,0,0,0,3,8,...,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,20
1,0,0,0,0,0,0,0,0,7,8,...,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,23
2,0,0,0,0,0,0,0,0,2,6,...,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,8
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,"(Oh, dear, !, I, shall, be, late, !, ')",Carroll,4
4,0,0,0,0,0,0,0,0,5,18,...,0,0,0,0,0,0,0,"((, when, she, thought, it, over, afterwards, ...",Carroll,35


In [36]:
print(np.where(combined2.columns == 'text_source'))

(array([2709, 3040], dtype=int64),)


In [37]:
# Since there are two columns for 'text_source,' drop one of them
combined2 = combined2.groupby(level=0, axis=1).last()

In [38]:
print(np.where(combined2.columns == 'text_source'))

(array([2709], dtype=int64),)


In [39]:
Y = combined2['text_source']
X = np.array(combined2.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [40]:
Y = combined2['text_source']
X = np.array(combined2.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = lr.fit(X_train, y_train)

print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

Training set score: 0.962922173275

Test set score: 0.94218061674


Adding parts of speech did not improve accuracy (previous was .935)

In [41]:
# Challenge 0
from sklearn.svm import SVC

# Trying out SVC
# Make a model using SVC here
svc = SVC()
svc.fit(X_train, y_train)

# Model.
print('\nTest set score:', svc.score(X_test, y_test))
svc_predicted = svc.predict(X_test)
pd.crosstab(y_test, svc_predicted)


Test set score: 0.751101321586


col_0,Austen,Carroll
text_source,,
Austen,1329,4
Carroll,448,35


In [42]:
svc = SVC(C=0.5, kernel='linear')
svc.fit(X_train, y_train)

print('\nTest set score:', svc.score(X_test, y_test))
svc_predicted = svc.predict(X_test)
pd.crosstab(y_test, svc_predicted)


Test set score: 0.938325991189


col_0,Austen,Carroll
text_source,,
Austen,1320,13
Carroll,99,384


# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.
Record your work for each challenge in a notebook and submit it below.

In [43]:
# list of books
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [44]:
# We will run the model for Austen's persusasion vs Shakespeare's Hamlet
hamlet = gutenberg.raw('shakespeare-hamlet.txt')

In [45]:
hamlet = re.sub(r'CHAPTER .*', '', hamlet)
#print(hamlet)

In [46]:
hamlet = text_cleaner(hamlet)

In [47]:
# Parse the cleaned novels. This can take a bit.
hamlet_doc = nlp(hamlet)

In [48]:
# Group into sentences.
hamlet_sents = [[sent, "Shakespeare"] for sent in hamlet_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(hamlet_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Actus, Primus, ., Scoena, Prima, .)",Shakespeare
1,"(Enter, Barnardo, and, Francisco, two, Centine...",Shakespeare
2,"(Barnardo, .)",Shakespeare
3,"(Who, 's, there, ?)",Shakespeare
4,"(Fran, .)",Shakespeare


In [49]:
# Set up the bags.
hamletwords = bag_of_words(hamlet_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words2 = set(alicewords + persuasionwords)

In [50]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words2)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
Processing row 5500


,send,flustered,jury,ann,signify,camden,pocket,dead,apparently,marry,...,style,constancy,stile,sign,mile,coaxing,loss,d,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Actus, Primus, ., Scoena, Prima, .)",Shakespeare
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Enter, Barnardo, and, Francisco, two, Centine...",Shakespeare
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Barnardo, .)",Shakespeare
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Who, 's, there, ?)",Shakespeare
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Fran, .)",Shakespeare


In [51]:
list(word_counts)

['send',
 'flustered',
 'jury',
 'ann',
 'signify',
 'camden',
 'pocket',
 'dead',
 'apparently',
 'marry',
 'crossly',
 "ma'am",
 'mamma',
 'bring',
 'extremely',
 'plain',
 'unfeeling',
 'responsible',
 'jug',
 'bid',
 'journey',
 'disposition',
 'dreamy',
 'conference',
 'footing',
 'lazy',
 'totally',
 'thatch',
 'idiotic',
 'monday',
 'seal',
 'husband',
 'late',
 'mabel',
 'attraction',
 'dear',
 'convenience',
 'gentleness',
 'advance',
 'mournful',
 'impression',
 'fatigue',
 'news',
 'correct',
 'interval',
 'clothe',
 'brother',
 'interested',
 "slates'll",
 'omit',
 'song',
 'climb',
 'bother',
 'kindly',
 'musgroves',
 'pay',
 'conceal',
 'conger',
 'heel',
 'distraction',
 'light',
 'possible',
 'recover',
 'twelfth',
 'straighten',
 'unhappy',
 'vague',
 'friendly',
 'coax',
 'rain',
 'risk',
 'confused',
 'heavy',
 'calculation',
 'except',
 'name',
 'favourite',
 'impulse',
 'later',
 'inquiry',
 'lovely',
 'refusal',
 'dish',
 'neatly',
 'feeble',
 'gentleman',
 'corre

### Due to the absence of crazy shakespeare-era words, it appears as though taking the lemmas of Hamlet yields words that are similar to Persuasion.

In [52]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [53]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3438, 3023) (3438,)
Training set score: 0.946189645143

Test set score: 0.879197557785


### The lr model was still good at predicting whether the sentence was written by Carroll or Shakespeare.